# Classificação com base em Regex

Como baseline, vamos buscar classificar as reviews em classes. Para isso, vamos primeiro identificar qual o tópico do qual a review trata (pode ser mais de um) a partir da identificação de palavaras (regex). 

Dessa forma, primeiramente foram definidas os seguintes tópicos que uma review pode falar, e as respectivas palavras que identificam esse tópico:

- Entrega do produto no sentido temporal 

    ['prazo', 'entrega', 'atraso', 'rápido', 'antes', 'estipulado', 'data', 'frete',
         'tempo', 'ainda', 'chegou', 'demorado', 'rapidez']

- Estado do produto ao chegar

    embalagem = ['danificado', 'embalado', 'lacre', 'rompido', 'estragado', 'quebrado', 'pacote', 'amassado',
            'caixa', 'vedada', 'intacto']

- Atendimento da loja

    atendimento = ['responder', 'responderam', 'contato', 'retornar', 'explicação', 'reclamação', 'loja', 
               'vendedor', 'americanas', 'atendimento']

- qualidade do produto

    produto = ['qualidade', 'ótimo', 'perfeito', 'bonito', 'interessante', 'amei', 'pessimo', 'fraco',
           'som', 'design', 'benefício', 'funciona', 'excelente', 'regular', 'caro', 'barato', 'preço']

A lista atualizada de palavras que categorizam cada tópico pode ser encontrada na pasta resources. A partir da topificação das reviews podemos definir a classe como sendo a combinação dos tópicos identificados

In [1]:
import os

os.chdir('../../../')
os.getcwd()

'/Users/brunogutierrez/workspace/tese'

Objetivo: comparar classificação com e sem stemmização

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from src.Dataloaders.b2w_dataset import B2W
from src.text_preprocessing import TextPreprocessing
from src.text_representation import *
from src.regex_review_classification import *

In [3]:
%%time
b2w = B2W()
b2w.pre_process()
df = b2w.df.sample(frac=0.2)

2023 linhas deletadas por terem menos de 3 palavras
2179 linhas deletadas por terem reviews iguais
CPU times: user 3.17 s, sys: 426 ms, total: 3.59 s
Wall time: 4.23 s


## Pré Processamento do texto

In [4]:
text_preprocessor = TextPreprocessing(df.copy(), 'full_review')
text_preprocessor_no_stemming = TextPreprocessing(df.copy(), 'full_review')

In [6]:
%%time
df_stem = text_preprocessor.pre_process()
df = text_preprocessor_no_stemming.pre_process(stemming=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brunogutierrez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


CPU times: user 18.4 s, sys: 200 ms, total: 18.6 s
Wall time: 20.4 s


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brunogutierrez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
 df_stem['clean_text'].sample(3)

35428                  bom gost recomendoentreg bastant rap
59432         parabens cheg bem rap original tud cert recom
110774    excepcional estad arte softwar escritori tud v...
Name: clean_text, dtype: object

In [8]:
 df['clean_text'].sample(3)

97991    tinta magenta nao funcionou completar tanques ...
91350    perfeita bom otimo acabamentobbbbbbbbbbbbbbbbb...
31436    nao compre marca microondas estragou meses uso...
Name: clean_text, dtype: object

### Classificação com base em regex

Podemos fazer a classificação de cada review a partir da identificação das palavras que denotam cada classe. Podemos fazer usando stemming ou não.

In [9]:
df['clean_text'].sample()

412    atrasado encontra atrasado demais ter chegado ...
Name: clean_text, dtype: object

In [10]:
df = regex_classification(df.copy(), stemming=False)
df['review_class'].value_counts()

produto                                7565
Nao identificado                       5763
produto-prazo                          5020
prazo                                  3885
atendimento-produto-prazo               735
atendimento-prazo                       643
atendimento                             469
atendimento-produto                     421
produto-prazo-embalagem                 290
prazo-embalagem                         235
produto-embalagem                       231
embalagem                               191
atendimento-produto-prazo-embalagem      61
atendimento-prazo-embalagem              50
atendimento-embalagem                    37
atendimento-produto-embalagem            24
Name: review_class, dtype: int64

In [11]:
df_stem['clean_text'].sample()

33655    nao compr variaca prec ness red indecent r vi ...
Name: clean_text, dtype: object

In [12]:
df_stem = regex_classification(df_stem.copy())
df_stem['review_class'].value_counts()

produto                                7491
produto-prazo                          5565
Nao identificado                       3823
prazo                                  2777
atendimento-produto-prazo              1531
atendimento-produto                    1355
atendimento                             689
atendimento-prazo                       545
produto-prazo-embalagem                 521
produto-embalagem                       465
embalagem                               273
prazo-embalagem                         242
atendimento-produto-prazo-embalagem     164
atendimento-produto-embalagem            82
atendimento-prazo-embalagem              51
atendimento-embalagem                    46
Name: review_class, dtype: int64